In [89]:
import pandas as pd
import datetime

pd.set_option('mode.chained_assignment', None) # remove copy warning

In [90]:
# get first infection dates
df = pd.read_csv('data/infections_timeseries.csv')
df = df.T
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df = df.drop(df.index[0])
df = df.ne(0).idxmax()
df_first = df.to_frame(name='FirstInfection')

In [91]:
df = pd.read_csv('data/infections_timeseries.csv', index_col = "FIPS")

# df = pd.read_csv('data/infections_timeseries_difference.csv', index_col = "FIPS") #here

df = df.drop('Combined_Key', axis = 1) # remove row

In [92]:
# get date of first infection first each county
df = df.stack()
df = df.to_frame()
df.reset_index(level=1, inplace=True)
df.columns = ['date', 'infected']
df

,date,infected
FIPS,,
1001,1/22/20,0
1001,1/23/20,0
1001,1/24/20,0
1001,1/25/20,0
1001,1/26/20,0
...,...,...
56045,5/29/20,1
56045,5/30/20,1
56045,5/31/20,1


In [93]:
# df.to_csv('times_series_transposed.csv')

In [94]:
# merge first infection date on infectiontimeseries 
df = pd.merge(df, df_first, left_index=True, right_index=True, how = 'left')
df

,date,infected,FirstInfection
FIPS,,,
1001,1/22/20,0,3/24/20
1001,1/23/20,0,3/24/20
1001,1/24/20,0,3/24/20
1001,1/25/20,0,3/24/20
1001,1/26/20,0,3/24/20
...,...,...,...
56045,5/29/20,1,5/27/20
56045,5/30/20,1,5/27/20
56045,5/31/20,1,5/27/20


In [95]:
# convert ordinal date's to regular date's
df_interventions = pd.read_csv('data/interventions.csv', index_col = "FIPS")

for index, row in df_interventions.iterrows(): # recode dates to normal format
    if pd.notna(row['stay at home']):
        df_interventions.loc[index, 'stay at home'] = datetime.date.fromordinal(int(row['stay at home']))
    if pd.notna(row['>50 gatherings']):
        df_interventions.loc[index, '>50 gatherings'] = datetime.date.fromordinal(int(row['>50 gatherings']))
    if pd.notna(row['>500 gatherings']):
        df_interventions.loc[index, '>500 gatherings'] = datetime.date.fromordinal(int(row['>500 gatherings']))
    if pd.notna(row['restaurant dine-in']):
        df_interventions.loc[index, 'restaurant dine-in'] = datetime.date.fromordinal(int(row['restaurant dine-in']))
    if pd.notna(row['public schools']):
        df_interventions.loc[index, 'public schools'] = datetime.date.fromordinal(int(row['public schools']))
    if pd.notna(row['entertainment/gym']):
        df_interventions.loc[index, 'entertainment/gym'] = datetime.date.fromordinal(int(row['entertainment/gym']))
    if pd.notna(row['federal guidelines']):
        df_interventions.loc[index, 'federal guidelines'] = datetime.date.fromordinal(int(row['federal guidelines']))
    if pd.notna(row['foreign travel ban']):
        df_interventions.loc[index, 'foreign travel ban'] = datetime.date.fromordinal(int(row['foreign travel ban']))

In [96]:
df_interventions

,STATE,AREA_NAME,stay at home,>50 gatherings,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban
FIPS,,,,,,,,,,
0,US,united states,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,AL,alabama,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
1001,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
1003,AL,baldwin county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
1005,AL,barbour county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2020-03-16,2020-03-11
...,...,...,...,...,...,...,...,...,...,...
72145,PR,"vega baja municipio, puerto rico",NaN,NaN,NaN,2020-03-16,2020-03-15,2020-03-15,2020-03-16,2020-03-11
72147,PR,"vieques municipio, puerto rico",NaN,NaN,NaN,2020-03-16,2020-03-15,2020-03-15,2020-03-16,2020-03-11
72149,PR,"villalba municipio, puerto rico",NaN,NaN,NaN,2020-03-16,2020-03-15,2020-03-15,2020-03-16,2020-03-11


In [97]:
# merge interventions with timeseries data
data = pd.merge(df, df_interventions, left_index=True, right_index=True, how = 'left')
data = data.drop(['federal guidelines', 'foreign travel ban'], axis = 1)
data.columns = ['date', 'infected', 'FirstInfection', 'STATE', 'AREA_NAME', 'home', 'gatherings50', 'gatherings500', 
                'schools', 'restaurant', 'entertainment']
data

,date,infected,FirstInfection,STATE,AREA_NAME,home,gatherings50,gatherings500,schools,restaurant,entertainment
FIPS,,,,,,,,,,,
1001,1/22/20,0,3/24/20,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
1001,1/23/20,0,3/24/20,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
1001,1/24/20,0,3/24/20,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
1001,1/25/20,0,3/24/20,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
1001,1/26/20,0,3/24/20,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28
...,...,...,...,...,...,...,...,...,...,...,...
56045,5/29/20,1,5/27/20,WY,weston county,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19
56045,5/30/20,1,5/27/20,WY,weston county,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19
56045,5/31/20,1,5/27/20,WY,weston county,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19


In [98]:
# data.to_csv('data.csv')

In [99]:
# convert data colums to datetime object 
data['date'] = pd.to_datetime(data['date'])
data['FirstInfection'] = pd.to_datetime(data['FirstInfection'])
data['home'] = pd.to_datetime(data['home'])
data['gatherings50'] = pd.to_datetime(data['gatherings50'])
data['gatherings500'] = pd.to_datetime(data['gatherings500'])
data['schools'] = pd.to_datetime(data['schools'])
data['restaurant'] = pd.to_datetime(data['restaurant'])
data['entertainment'] = pd.to_datetime(data['entertainment'])

data['difference'] = (data['date'] - data['FirstInfection']).dt.days
data

,date,infected,FirstInfection,STATE,AREA_NAME,home,gatherings50,gatherings500,schools,restaurant,entertainment,difference
FIPS,,,,,,,,,,,,
1001,2020-01-22,0,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,-62
1001,2020-01-23,0,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,-61
1001,2020-01-24,0,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,-60
1001,2020-01-25,0,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,-59
1001,2020-01-26,0,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,-58
...,...,...,...,...,...,...,...,...,...,...,...,...
56045,2020-05-29,1,2020-05-27,WY,weston county,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19,2
56045,2020-05-30,1,2020-05-27,WY,weston county,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19,3
56045,2020-05-31,1,2020-05-27,WY,weston county,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19,4


In [100]:
# get first two weeks after first infection was registered
# data = data[data['difference'] >= 0]
# data = data[data['difference'] < 14]

# get first 60 days, every five days
data = data[data['difference'] >= 0]
data = data[data['difference'] < 14]
# data = data[data['difference'].mod(3) == 0]

display(data)

,date,infected,FirstInfection,STATE,AREA_NAME,home,gatherings50,gatherings500,schools,restaurant,entertainment,difference
FIPS,,,,,,,,,,,,
1001,2020-03-24,1,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,0
1001,2020-03-25,4,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,1
1001,2020-03-26,6,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,2
1001,2020-03-27,6,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,3
1001,2020-03-28,6,2020-03-24,AL,autauga county,2020-04-04,2020-03-20,2020-03-13,2020-03-16,2020-03-19,2020-03-28,4
...,...,...,...,...,...,...,...,...,...,...,...,...
56045,2020-05-29,1,2020-05-27,WY,weston county,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19,2
56045,2020-05-30,1,2020-05-27,WY,weston county,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19,3
56045,2020-05-31,1,2020-05-27,WY,weston county,2020-03-28,2020-03-20,2020-03-20,2020-03-20,2020-03-19,2020-03-19,4


In [101]:
# get date differences
data['home'] = (data['date'] - data['home']).dt.days
data['gatherings50'] = (data['date'] - data['gatherings50']).dt.days
data['gatherings500'] = (data['date'] - data['gatherings500']).dt.days
data['schools'] = (data['date'] - data['schools']).dt.days
data['restaurant'] = (data['date'] - data['restaurant']).dt.days
data['entertainment'] = (data['date'] - data['entertainment']).dt.days

In [102]:
# data.reset_index(level=0, inplace=True)
data
data1 = data.copy()
data1 = data1.fillna(0) 

In [103]:
data1['home'] = data1['home']
data1['gatherings50'] = data1['gatherings50']
data1['gatherings500'] = data1['gatherings500']
data1['schools'] = data1['schools']
data1['restaurant'] = data1['restaurant']
data1['entertainment'] =data1['entertainment']

In [104]:
# set interventions to 0 if not implemented and 1 if implemented
data1['home'][data1['home']  < 0] = 0
data1['gatherings50'][data1['gatherings50']  < 0] = 0
data1['gatherings500'][data1['gatherings500']  < 0] = 0
data1['schools'][data1['schools']  < 0] = 0
data1['restaurant'][data1['restaurant']  < 0] = 0
data1['entertainment'][data1['entertainment']  < 0] = 0

data1['home'][data1['home']  > 0] = 1
data1['gatherings50'][data1['gatherings50']  > 0] = 1
data1['gatherings500'][data1['gatherings500']  > 0] = 1
data1['schools'][data1['schools']  > 0] = 1
data1['restaurant'][data1['restaurant']  > 0] = 1
data1['entertainment'][data1['entertainment']  > 0] = 1

In [105]:
data1

,date,infected,FirstInfection,STATE,AREA_NAME,home,gatherings50,gatherings500,schools,restaurant,entertainment,difference
FIPS,,,,,,,,,,,,
1001,2020-03-24,1,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,0
1001,2020-03-25,4,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,1
1001,2020-03-26,6,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,2
1001,2020-03-27,6,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,3
1001,2020-03-28,6,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
56045,2020-05-29,1,2020-05-27,WY,weston county,1.0,1.0,1.0,1,1,1.0,2
56045,2020-05-30,1,2020-05-27,WY,weston county,1.0,1.0,1.0,1,1,1.0,3
56045,2020-05-31,1,2020-05-27,WY,weston county,1.0,1.0,1.0,1,1,1.0,4


In [106]:
# data1['difference'] = data1['difference'] + 1 # make time start at t=1
# data1.to_csv('panel_data.csv')

In [107]:
# set index to column
panel_data = data1
panel_data = panel_data.reset_index(level=0)
panel_data

,FIPS,date,infected,FirstInfection,STATE,AREA_NAME,home,gatherings50,gatherings500,schools,restaurant,entertainment,difference
0,1001,2020-03-24,1,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,0
1,1001,2020-03-25,4,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,1
2,1001,2020-03-26,6,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,2
3,1001,2020-03-27,6,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,3
4,1001,2020-03-28,6,2020-03-24,AL,autauga county,0.0,1.0,1.0,1,1,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43739,56045,2020-05-29,1,2020-05-27,WY,weston county,1.0,1.0,1.0,1,1,1.0,2
43740,56045,2020-05-30,1,2020-05-27,WY,weston county,1.0,1.0,1.0,1,1,1.0,3
43741,56045,2020-05-31,1,2020-05-27,WY,weston county,1.0,1.0,1.0,1,1,1.0,4
43742,56045,2020-06-01,1,2020-05-27,WY,weston county,1.0,1.0,1.0,1,1,1.0,5


In [108]:
#create balanced dataframe
unbalanced = panel_data.groupby('FIPS')['date'].count()
unbalanced = unbalanced.to_frame()
display(unbalanced)
unbalanced = unbalanced[unbalanced['date'] != 14]
# unbalanced = unbalanced.reset_index(level=0)
unbalanced = unbalanced.index.tolist()
unbalanced
panel_data_balanced = panel_data[~panel_data['FIPS'].isin(unbalanced)]
panel_data_balanced = panel_data_balanced.dropna()

print(panel_data.shape)
print(panel_data_balanced.shape)

,date
FIPS,
1001,14
1003,14
1005,14
1007,14
1009,14
...,...
56037,14
56039,14
56041,14


(43744, 13)
(43484, 13)


In [109]:
# panel_data_balanced.to_csv('panel_data_balanced1.csv')

In [110]:
county_data = pd.read_csv('data/counties.csv')
county_data = county_data[['FIPS', 'Percent of adults completing some college or associate\'s degree 2014-18',
                           'Median_Household_Income_2018', 'Apr Temp AVG / F', 
                           'Density per square mile of land area - Population', 'Total_age65plus', 
                           'Active General Surgeons per 100000 Population 2018 (AAMC)']]
county_data.columns = ['FIPS', 'college_degree', 'HH_income', 'TempApr', 'PP_density', 'ageabove65', 'General_surgeons']

In [111]:
county_data

,FIPS,college_degree,HH_income,TempApr,PP_density,ageabove65,General_surgeons
0,0,29.0,61937.0,NaN,87.4,52431193.0,NaN
1,1000,29.9,49881.0,63.26,94.4,NaN,7.6
2,1001,28.4,59338.0,65.00,91.8,8653.0,7.6
3,1003,31.3,57588.0,66.00,114.6,44571.0,7.6
4,1005,25.1,34382.0,64.90,31.0,4832.0,7.6
...,...,...,...,...,...,...,...
3268,72145,25.2,NaN,NaN,1301.1,NaN,NaN
3269,72147,17.0,NaN,NaN,183.2,NaN,NaN
3270,72149,18.7,NaN,NaN,731.6,NaN,NaN
3271,72151,26.3,NaN,NaN,687.2,NaN,NaN


In [112]:
panel_data_control = pd.merge(panel_data_balanced, county_data, on = 'FIPS', how = 'left')
panel_data_control.shape

(43484, 19)

In [113]:
panel_data_control = panel_data_control.dropna()
panel_data_control.shape

(43372, 19)

In [114]:
panel_data_control['difference'] = panel_data_control['difference'] + 1

In [119]:
import numpy as np

# panel_data_control['infectedT'] = round(np.log(1 + panel_data_control['infected'] / panel_data_control['PP_density']), 5)
panel_data_control.to_csv('data_edited/panel_data_control14_edited.csv')

In [116]:
# panel_data_contßΩrol['infectedT'].dtype

In [117]:
# panel_data_control['infected_diff'] = panel_data_control['infected'].diff(periods=-1)

In [118]:
# panel_data_control